In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
import pandas_ta as ta

from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from functools import reduce

from sklearn.model_selection import GridSearchCV

In [2]:
def get_data(ticker, year,month,day):
    start = dt.datetime(1999,1,1)
    end = dt.datetime(2023,8,29)
     

    df = yf.download(ticker,start,end)
    
    return df

In [3]:
# loop through dataframe and add all features to a list
# removes 'Tomorrow' and 'Target' column from the list 
def get_features(dataframe):
    features = []
    for column in dataframe:
        features.append(column)
    
    features.pop(-1)
    features.pop(-1)
    
    return features

In [4]:
def fit_train_score(df, n_est, min_split):    
    features = get_features(df)
    
    # n_estimators = number of decision trees
    # min_samples_split = higher it is set, the less accurtate it is, the less it will overfit
    model = RandomForestClassifier(n_estimators=n_est, min_samples_split=min_split, random_state=1)

    train = df.iloc[:-1000]
    test = df.iloc[-1000:]

    # fit and train model
    model.fit(train[features], train['Target'])
    predictions = model.predict(test[features])
    #predictions_series = pd.Series(predictions, index=test.index)

    # precision of training data
    predictions_training = model.predict(train[features])
    
    test_dict = {
        'n_estimators':n_est,
        'min_samples_split':min_split,
        'training precision':precision_score(train['Target'], predictions_training),
        'testing precision':precision_score(test['Target'], predictions)
                }
    
    return test_dict

In [5]:
# same as fit_train_score but it excepts arguments for max_depth
def fit_train_score_with_depth(df, n_est, min_split, max_depth):
    features = get_features(df)
    
    # n_estimators = number of decision trees
    # min_samples_split = higher it is set, the less accurtate it is, the less it will overfit
    model = RandomForestClassifier(n_estimators=n_est, min_samples_split=min_split, random_state=1)

    train = df.iloc[:-1000]
    test = df.iloc[-1000:]

    # fit and train model
    model.fit(train[features], train['Target'])
    predictions = model.predict(test[features])
    #predictions_series = pd.Series(predictions, index=test.index)

    # precision of training data
    predictions_training = model.predict(train[features])
    
    test_dict = {
        'n_estimators':n_est,
        'min_samples_split':min_split,
        'max_depth': max_depth,
        'training precision':precision_score(train['Target'], predictions_training),
        'testing precision':precision_score(test['Target'], predictions)
                }
    
    return test_dict

In [6]:
# n_est, min_sample, and max_depth should be lists with values you want to test
def optimize_hyperparams(df, n_est, min_sample, max_depth):
    features = get_features(df)
    
    param_grid = { 
        'n_estimators': n_est,
        'min_samples_split': min_sample,
        'max_depth' : max_depth,
    }

    model = RandomForestClassifier(random_state=1)

    train = df.iloc[:-1000]
    test = df.iloc[-1000:]

    CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
    CV_rfc.fit(train[features], train['Target'])
    
    print(CV_rfc.best_params_)

In [7]:
df = get_data('SPY',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [8]:
# create new features
df['Range'] = abs(df['High']-df['Low'])
df['RSI 14'] = ta.rsi(df['Close'], 14)
df['SMA 20'] = ta.sma(df['Close'], 20)
df['SMA 50'] = ta.sma(df['Close'], 50)
df['SMA 200'] = ta.sma(df['Close'], 200)
df['EMA 12'] = ta.ema(df['Close'], 12)
df['EMA 26'] = ta.ema(df['Close'], 26)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'] )

# create 'Tomorrow' column
# it's the 'Close' value from the previous day
# will be used to create a target
df['Tomorrow'] = df['Close'].shift(-1)
df['Target'] = (df['Tomorrow'] > df['Close']).astype(int)
df = df.loc['2000-01-01':].copy()
df.drop('Adj Close', axis=1, inplace=True)

# save most recent daily data
#df_most_recent = pd.DataFrame(df.tail(1))

# drop most recent day from dataframe
#df.drop(df.tail(1).index, inplace=True)

In [9]:
len(df)

5951

In [10]:
# get foreign market and gold data
UK_df = get_data('^FTSE',1999,1,1)
China_df = get_data('000001.SS',1999,1,1)
Germany_df = get_data('^GDAXI',1999,1,1)
Japan_df = get_data('^N225',1999,1,1)
gold_df = get_data('GC=F',1999,1,1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
print(len(UK_df))
print(len(China_df))
print(len(Germany_df))
print(len(Japan_df))
print(len(gold_df))

6225
5965
6260
6041
5768


In [12]:
# clean foreign market data
# some have empty volume columns for early 2000's

# drop volume from foreign data dfs
#rename columns
# remove first year of dat

UK_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
UK_df.rename(columns={
    'Open':'UK Open',
    'High':'UK High',
    'Low':'UK Low',
    'Close':'UK Close'
},inplace=True)
UK_df.loc['2000-01-01':]

China_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
China_df.rename(columns={
    'Open':'China Open',
    'High':'China High',
    'Low':'China Low',
    'Close':'China Close'
},inplace=True)
China_df.loc['2000-01-01':]

Germany_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
Germany_df.rename(columns={
    'Open':'Germany Open',
    'High':'Germany High',
    'Low':'Germany Low',
    'Close':'Germany Close'
},inplace=True)
Germany_df.loc['2000-01-01':]

Japan_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
Japan_df.rename(columns={
    'Open':'Japan Open',
    'High':'Japan High',
    'Low':'Japan Low',
    'Close':'Japan Close'
},inplace=True)
Japan_df.loc['2000-01-01':]

gold_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
gold_df.rename(columns={
    'Open':'Gold Open',
    'High':'Gold High',
    'Low':'Gold Low',
    'Close':'Gold Close'
},inplace=True)
gold_df.loc['2000-01-01':]

,Gold Open,Gold High,Gold Low,Gold Close
Date,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994
2000-08-31,274.799988,278.299988,274.799988,278.299988
2000-09-01,277.000000,277.000000,277.000000,277.000000
2000-09-05,275.799988,275.799988,275.799988,275.799988
2000-09-06,274.200012,274.200012,274.200012,274.200012
...,...,...,...,...
2023-08-22,1894.599976,1896.800049,1892.800049,1896.400024
2023-08-23,1909.599976,1918.500000,1909.599976,1918.500000
2023-08-24,1920.000000,1920.800049,1918.199951,1918.199951


In [13]:
print(len(UK_df))
print(len(China_df))
print(len(Germany_df))
print(len(Japan_df))
print(len(gold_df))

6225
5965
6260
6041
5768


In [31]:
# merge into one df
dfs = [UK_df, China_df, Germany_df, Japan_df, gold_df, df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), dfs)

# drop most recent day from dataframe
df_merged.drop(df_merged.tail(1).index, inplace=True)

df_merged

,UK Open,UK High,UK Low,UK Close,China Open,China High,China Low,China Close,Germany Open,Germany High,...,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-30,6586.299805,6624.899902,6586.100098,6615.100098,2107.019043,2108.096924,2075.159912,2076.143066,7292.459961,7292.459961,...,1.156250,55.725080,149.267188,147.763125,144.410859,150.345132,149.168169,1.661060,152.343750,1
2000-08-31,6615.100098,6675.700195,6585.200195,6672.700195,2062.834961,2064.760010,2019.447021,2021.198975,7191.000000,7251.310059,...,2.187500,62.639763,149.604688,147.852500,144.466328,150.652612,149.403397,1.738841,152.500000,1
2000-09-01,6672.700195,6838.600098,6672.700195,6795.000000,2009.151001,2026.350952,1997.717041,1999.859009,7221.410156,7397.120117,...,1.593750,63.124303,149.910937,147.990000,144.520703,150.936826,149.632775,1.728478,151.281250,0
2000-09-05,6798.100098,6809.500000,6737.600098,6752.500000,2015.062012,2020.847046,1959.592041,1963.791992,7448.080078,7456.709961,...,1.390625,56.922949,150.068750,148.128125,144.563984,150.989814,149.754884,1.725551,149.562500,0
2000-09-06,6752.500000,6766.000000,6688.399902,6694.700195,1955.499023,1964.265015,1923.598022,1925.906982,7397.049805,7434.020020,...,2.421875,49.532617,150.112500,148.194687,144.599297,150.770227,149.740634,1.775288,150.843750,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-18,7310.200195,7310.200195,7215.799805,7262.399902,3165.100098,3179.120117,3131.949951,3131.949951,15597.120117,15612.879883,...,4.559998,35.283750,448.740503,444.149401,412.000000,443.726081,445.381265,4.519322,439.339996,1
2023-08-21,7262.399902,7316.000000,7250.899902,7257.799805,3125.989990,3134.959961,3092.979980,3092.979980,15573.950195,15710.209961,...,4.789978,40.887683,447.997502,444.353600,412.274100,443.051299,444.933763,4.538655,438.149994,0
2023-08-22,7257.799805,7310.500000,7257.600098,7270.799805,3103.780029,3126.780029,3074.530029,3120.330078,15684.639648,15799.339844,...,3.609985,39.350095,447.133002,444.518600,412.590500,442.297252,444.431262,4.472321,443.029999,1


In [22]:
df_merged.tail()

,UK Open,UK High,UK Low,UK Close,China Open,China High,China Low,China Close,Germany Open,Germany High,...,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,,,,,,,
2010-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.919998,72.321706,116.448500,112.404800,106.433400,117.044569,115.583474,1.211515,119.040001,1.0
2014-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.239990,50.722610,207.396001,207.433401,201.454800,207.225337,207.583408,2.162638,207.279999,0.0
2017-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.970001,62.029661,235.722999,236.194399,224.041400,236.940016,236.096638,1.600913,238.770004,1.0
2021-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.559998,67.670551,393.285001,388.377801,355.071351,396.012714,392.516068,5.233052,406.119995,0.0


In [21]:
df_merged.count()

UK Open          6225
UK High          6225
UK Low           6225
UK Close         6225
China Open       5965
China High       5965
China Low        5965
China Close      5965
Germany Open     6260
Germany High     6260
Germany Low      6260
Germany Close    6260
Japan Open       6040
Japan High       6040
Japan Low        6040
Japan Close      6040
Gold Open        5767
Gold High        5767
Gold Low         5767
Gold Close       5767
Open             5950
High             5950
Low              5950
Close            5950
Volume           5950
Range            5950
RSI 14           5950
SMA 20           5950
SMA 50           5950
SMA 200          5950
EMA 12           5950
EMA 26           5950
ATR              5950
Tomorrow         5949
Target           5950
dtype: int64

In [32]:
df_merged.dropna(inplace=True)
df_merged.count()

UK Open          4992
UK High          4992
UK Low           4992
UK Close         4992
China Open       4992
China High       4992
China Low        4992
China Close      4992
Germany Open     4992
Germany High     4992
Germany Low      4992
Germany Close    4992
Japan Open       4992
Japan High       4992
Japan Low        4992
Japan Close      4992
Gold Open        4992
Gold High        4992
Gold Low         4992
Gold Close       4992
Open             4992
High             4992
Low              4992
Close            4992
Volume           4992
Range            4992
RSI 14           4992
SMA 20           4992
SMA 50           4992
SMA 200          4992
EMA 12           4992
EMA 26           4992
ATR              4992
Tomorrow         4992
Target           4992
dtype: int64

In [34]:
df_merged.tail()

,UK Open,UK High,UK Low,UK Close,China Open,China High,China Low,China Close,Germany Open,Germany High,...,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-18,7310.200195,7310.200195,7215.799805,7262.399902,3165.100098,3179.120117,3131.949951,3131.949951,15597.120117,15612.879883,...,4.559998,35.283750,448.740503,444.149401,412.0000,443.726081,445.381265,4.519322,439.339996,1
2023-08-21,7262.399902,7316.000000,7250.899902,7257.799805,3125.989990,3134.959961,3092.979980,3092.979980,15573.950195,15710.209961,...,4.789978,40.887683,447.997502,444.353600,412.2741,443.051299,444.933763,4.538655,438.149994,0
2023-08-22,7257.799805,7310.500000,7257.600098,7270.799805,3103.780029,3126.780029,3074.530029,3120.330078,15684.639648,15799.339844,...,3.609985,39.350095,447.133002,444.518600,412.5905,442.297252,444.431262,4.472321,443.029999,1
2023-08-23,7270.799805,7348.799805,7270.799805,7320.500000,3116.270020,3116.379883,3077.610107,3078.399902,15761.250000,15820.950195,...,4.570007,47.987997,446.509001,444.703201,412.9506,442.409982,444.327464,4.547157,436.890015,0
2023-08-24,7320.500000,7386.100098,7320.500000,7333.600098,3085.909912,3106.179932,3072.949951,3082.239990,15876.219727,15896.740234,...,8.360016,40.225352,445.729002,444.707801,413.2533,441.560757,443.776542,4.819504,439.970001,1


In [ ]:
fit_train_score(df_merged,200,750)

In [ ]:
fit_train_score(df_merged,200,650)

In [93]:
def optimize_hyperparams(df, n_est, min_sample, max_depth):
    features = get_features(df)
    
    param_grid = { 
        'n_estimators': n_est,
        'min_samples_split': min_sample,
        'max_depth' : max_depth,
    }

    model = RandomForestClassifier(random_state=1)

    train = df.iloc[:-1000]
    test = df.iloc[-1000:]

    CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
    CV_rfc.fit(train[features], train['Target'])
    
    print(CV_rfc.best_params_)

In [94]:
n_est = [200,250,300,350,400]
min_samples_split = [400, 450, 500, 550, 600,650,700]
max_depth = [4,5,6,7,8]

# uses GridSearchCV from sklearn to find best hyperparameters
# can take a VERY long time to run if there are a lot of hyperparameters to loop through
optimize_hyperparams(df_merged, n_est, min_samples_split, max_depth)

{'max_depth': 4, 'min_samples_split': 700, 'n_estimators': 350}


In [25]:
# using optimized hyperparameters from above
fit_train_score_with_depth(df, 700, 350, 4)

{'n_estimators': 700,
 'min_samples_split': 350,
 'max_depth': 4,
 'training precision': 0.6168662155127538,
 'testing precision': 0.5909090909090909}

In [19]:
df_most_recent = df_merged[-1:].copy()
df_most_recent

,UK Open,UK High,UK Low,UK Close,China Open,China High,China Low,China Close,Germany Open,Germany High,...,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-24,7320.5,7386.100098,7320.5,7333.600098,3085.909912,3106.179932,3072.949951,3082.23999,15876.219727,15896.740234,...,8.360016,40.225352,445.729002,444.707801,413.2533,441.560757,443.776542,4.819504,439.970001,1


In [ ]:
# check if model could correctly predict today 

features = get_features(df_merged)

# n_estimators = number of decision trees
# min_samples_split = higher it is set, the less accurtate it is, the less it will overfit
model = RandomForestClassifier(n_estimators=200, min_samples_split=400, max_depth=5, random_state=1)

train = df_merged.iloc[:-1000].copy()
test = df_merged.iloc[-1000:].copy()

# fit and train model
model.fit(train[features], train['Target'])
predictions = model.predict(df_most_recent[features])
#predictions_series = pd.Series(predictions, index=test.index)



print(predictions)

# model predict a down day, but Monday was an up day            

In [28]:
# model to use for backtesting
final_model = RandomForestClassifier(n_estimators=200, min_samples_split=400, max_depth=5, random_state=1)

In [27]:
# make predictions

# arguments for funtion
    # train = df_merged.iloc[:-1000].copy()
    # test = df_merged.iloc[-1000:].copy()
    # final_model
    # features = get_features(merged_df)
def predict(train, test, features, model):
    model.fit(train[features], train["Target"])
    preds = model.predict(test[features])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [29]:
def backtest(df, model, features, start=2500, step=250):
    all_predictions = []

    for i in range(start, df.shape[0]-step, step):
        train = df.iloc[0:i].copy()
        test = df.iloc[i:(i+step)].copy()
        predictions = predict(train, test, features, model)
        
        print(df.index[i])
        print(precision_score(predictions["Target"], predictions["Predictions"]))
        
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [30]:
predictions = backtest(df, final_model, get_features(df))

2009-12-10 00:00:00
0.5591836734693878
2010-12-08 00:00:00
0.5349794238683128
2011-12-05 00:00:00
0.5365853658536586
2012-12-04 00:00:00
0.5882352941176471
2013-12-02 00:00:00
0.6108597285067874
2014-11-28 00:00:00
0.484
2015-11-25 00:00:00
0.5514705882352942
2016-11-22 00:00:00
0.0


C:\Users\13148\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2017-11-20 00:00:00
0.536
2018-11-16 00:00:00
0.6
2019-11-15 00:00:00
0.588
2020-11-12 00:00:00
0.0


C:\Users\13148\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2021-11-10 00:00:00
0.452


In [ ]:
df_merged

In [41]:
most_recent = yf.download('SPY',pd.Timestamp.now())

[*********************100%***********************]  1 of 1 completed


In [42]:
most_recent

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-08-28,442.23999,443.399994,439.972809,442.76001,442.76001,56650963


In [96]:
model2 = RandomForestClassifier(n_estimators=700, min_samples_split=350, max_depth=4, random_state=1)

In [100]:
predictions2 = backtest(df_merged, model2, get_features(df_merged))

2012-03-07 00:00:00
0.5523809523809524
2013-05-08 00:00:00
0.5666666666666667
2014-06-30 00:00:00
0.516
2015-08-13 00:00:00
0.5380434782608695
2016-10-18 00:00:00
0.5333333333333333
2017-12-04 00:00:00
0.5602094240837696
2019-01-28 00:00:00
0.59375
2020-03-31 00:00:00
0.6415094339622641
2021-05-26 00:00:00
0.5193370165745856
